In [1]:
from logic.getting import *

In [2]:
from logic.scaler import *

In [3]:
from logic.windows_preproc import *

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [5]:
from tensorflow.keras.layers import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM, GRU, Embedding
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import hinge, Recall, Precision, PrecisionAtRecall
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.utils as utils
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [125]:
price = get_price_data_raw('NEE')

/Users/oscarthieleserrano/.pyenv/versions/3.10.6/envs/portfolio_manager/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [186]:
price

,datetime,open,high,low,close,volume
47665,2005-01-03 09:30:00+00:00,5.278,5.304,5.194,5.228,1110640
62557,2005-01-03 10:00:00+00:00,5.229,5.277,5.185,5.244,679280
30227,2005-01-03 10:30:00+00:00,5.245,5.273,5.191,5.223,556400
24130,2005-01-03 11:00:00+00:00,5.223,5.253,5.172,5.219,721040
69934,2005-01-03 11:30:00+00:00,5.220,5.249,5.186,5.216,479680
...,...,...,...,...,...,...
39723,2023-08-31 17:30:00+00:00,65.720,65.723,65.640,65.648,11179
39724,2023-08-31 18:00:00+00:00,65.720,65.723,65.689,65.707,1062
39726,2023-08-31 18:30:00+00:00,65.661,65.723,65.640,65.707,2294171
39727,2023-08-31 19:00:00+00:00,65.661,65.723,65.640,65.707,2293999


In [127]:
price = price.sort_values(by='timestamp_field_0')

In [128]:
tech = get_technical_analysis('NEE')

/Users/oscarthieleserrano/.pyenv/versions/3.10.6/envs/portfolio_manager/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [129]:
macro = get_macro_data()

In [130]:
tech = tech.sort_values(by='timestamp_field_0')

In [187]:
macro = macro.sort_values(by='timestamp')

In [188]:
def rename_macroeconomic(df_macroeconomic):
    df_macroeconomic = df_macroeconomic.drop(columns = "int64_field_0")
    columns_macroeconomic = ["datetime", "interest_rate", "GDP", "inflation"]
    df_macroeconomic.columns = columns_macroeconomic
    return df_macroeconomic

def merge_macroeconomic(df_technical, df_macroeconomic):
    df_merged = df_technical.merge(df_macroeconomic, on="datetime", how="inner")
    return df_merged

def rename_columns(df):
    columns = ["datetime", "open", "high", "low", "close", "volume"]
    df.columns = columns
    return df

def rename_technical(df_technical):
    df_technical = df_technical.rename(columns = {"timestamp_field_0": "datetime"})
    return df_technical

def convert_datetime(df):
    data = df.copy()
    data["datetime"] = pd.to_datetime(data["datetime"])
    return data

def merge_columns(df_values, df_technical):
    df_merged = df_values.merge(df_technical, how="inner", on="datetime")
    return df_merged

def clean_data(df):
    df_inter = df.interpolate(method='linear')
    df_clean = df_inter.dropna()
    return df_clean

def create_target(df, column_name, new_column_name='Target'):
    result = []
    
    for i in range(1, len(df)):
        change_percent = (df[column_name].iloc[i] / df[column_name].iloc[i-1] - 1) * 100
        if change_percent > 0.07:
            result.append(1)
        elif change_percent < -0.07:
            result.append(-1)
        else:
            result.append(0)
    result = [float('nan')] + result
    df[new_column_name] = pd.Series(result, index=df.index)
    
    return df

def target_drop(df_final):
    df_final = df_final.dropna()
    return df_final

def convert_time_sin_cos(df):
    data = df.copy()
    data['day_of_year'] = data['datetime'].dt.dayofyear
    data['time_of_day'] = data['datetime'].dt.hour * 3600 + data['datetime'].dt.minute * 60 + data['datetime'].dt.second
    data['day_of_year_norm'] = data['day_of_year'] / 365.0
    data['time_of_day_norm'] = data['time_of_day'] / 86400.0
    data['cos_time_of_day'] = np.cos(2 * np.pi * data['time_of_day_norm'])
    data['sin_time_of_day'] = np.sin(2 * np.pi * data['time_of_day_norm'])
    data["cos_day_of_year"] = np.cos(2 * np.pi * data['day_of_year_norm'])
    data["sin_day_of_year"] = np.sin(2 * np.pi * data['day_of_year_norm'])
    df_converted = data.drop(columns = ["day_of_year", "time_of_day", "day_of_year_norm", "time_of_day_norm"])
    return df_converted

def log_divide_next(df):
    log_returns = np.log(df['close'] / df['close'].shift(1))
    return log_returns

def split_into_categories(df, column_name, prefix='Category'):

    df[f'{prefix}_1'] = (df[column_name] == 1).astype(int)
    df[f'{prefix}_0'] = (df[column_name] == 0).astype(int)
    df[f'{prefix}_-1'] = (df[column_name] == -1).astype(int)
    
    return df

In [189]:
def create_x_y(df_data, df_technical, df_macroeconomic):
    df_macroeconomic = rename_macroeconomic(df_macroeconomic)
    df_technical = rename_technical(df_technical)
    df_data = rename_columns(df_data)
    df_data = df_data.sort_values(by="datetime")
    df_technical = df_technical.sort_values(by="datetime")
    df_macroeconomic = df_macroeconomic.sort_values(by="datetime")
    df_technical = clean_data(df_technical)
    df_data = convert_datetime(df_data)
    df_technical = convert_datetime(df_technical)
    df_macroeconomic = convert_datetime(df_macroeconomic)
    df_technical = merge_macroeconomic(df_technical, df_macroeconomic)
    df_merged = merge_columns(df_data, df_technical)
    df_target = create_target(df_merged, "close")
    df_final = target_drop(df_target)
    df_time = convert_time_sin_cos(df_final)
    X = df_time.drop(columns = "Target")
    y_initial = df_time[["Target"]]
    y = split_into_categories(y_initial, "Target")
    y = y.drop(columns = "Target")
    log_df = log_divide_next(df_data).dropna()
    return X, y, log_df

In [190]:
X, y, log_df = create_x_y(price, tech, macro)

/var/folders/7w/19z1p17n6vb_v1k8gw86dgm40000gn/T/ipykernel_15439/4139434303.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{prefix}_1'] = (df[column_name] == 1).astype(int)
/var/folders/7w/19z1p17n6vb_v1k8gw86dgm40000gn/T/ipykernel_15439/4139434303.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'{prefix}_0'] = (df[column_name] == 0).astype(int)
/var/folders/7w/19z1p17n6vb_v1k8gw86dgm40000gn/T/ipykernel_15439/4139434303.py:75: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [462]:
X


,open,high,low,close,volume,EMA,SlowK,SlowD,Real Upper Band,Real Middle Band,...,MACD,MACD_Signal,MACD_Hist,RSI,OBV,SAR,cos_time_of_day,sin_time_of_day,cos_day_of_year,sin_day_of_year
1,5.158,5.214,5.126,5.188,546400,5.1509,42.1391,41.6396,5.2491,5.1479,...,0.0077,0.0005,0.0072,40.9712,-2236160.0,5.2428,-9.659258e-01,2.588190e-01,0.992749,0.120208
2,5.187,5.217,5.144,5.176,347200,5.1521,55.0725,45.4493,5.2455,5.1469,...,0.0113,0.0027,0.0086,43.5344,-2583360.0,5.2396,-9.914449e-01,1.305262e-01,0.992749,0.120208
3,5.178,5.204,5.134,5.169,224800,5.1529,61.3778,52.8631,5.2431,5.1462,...,0.0133,0.0048,0.0085,42.9459,-2808160.0,5.2365,-1.000000e+00,1.224647e-16,0.992749,0.120208
4,5.172,5.202,5.129,5.159,288800,5.1535,58.7396,58.3966,5.2406,5.1454,...,0.0142,0.0067,0.0075,42.5558,-3096960.0,5.2335,-9.914449e-01,-1.305262e-01,0.992749,0.120208
5,5.160,5.189,5.127,5.162,269600,5.1538,53.0858,57.7344,5.2378,5.1444,...,0.0138,0.0081,0.0057,41.7840,-2827360.0,5.2305,-9.659258e-01,-2.588190e-01,0.992749,0.120208
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77392,65.720,65.723,65.640,65.648,11179,66.0978,19.4066,15.3858,66.4707,66.0719,...,-0.1047,-0.0357,-0.0690,46.5953,-18854724.0,66.6033,-1.305262e-01,-9.914449e-01,-0.504961,-0.863142
77393,65.720,65.723,65.689,65.707,1062,66.0854,44.1865,26.7183,66.4603,66.0606,...,-0.1122,-0.0510,-0.0612,46.5953,-18853662.0,66.5723,-1.836970e-16,-1.000000e+00,-0.504961,-0.863142
77394,65.661,65.723,65.640,65.707,2294171,66.0715,64.1516,42.5816,66.4488,66.0478,...,-0.1215,-0.0651,-0.0565,46.1644,-18853662.0,66.5423,1.305262e-01,-9.914449e-01,-0.504961,-0.863142
77395,65.661,65.723,65.640,65.707,2293999,66.0580,86.4690,64.9357,66.4552,66.0424,...,-0.1275,-0.0776,-0.0499,46.1644,-18853662.0,66.5132,2.588190e-01,-9.659258e-01,-0.504961,-0.863142


In [463]:
y

,Category_1,Category_0,Category_-1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
5,0,1,0
...,...,...,...
77392,0,0,1
77393,1,0,0
77394,0,1,0
77395,0,1,0


In [444]:
X = X.drop(columns=['GDP', 'inflation', 'interest_rate'])

KeyError: "['GDP', 'inflation', 'interest_rate'] not found in axis"

In [445]:
X = X.drop(columns='datetime')

KeyError: "['datetime'] not found in axis"

In [464]:
X_train = X[50000:72000]

In [465]:
X_val = X[72000:73000]

In [466]:
X_test = X[73000:]

In [467]:
y_train = y[50000:72000]

In [468]:
y_val = y[72000:73000]

In [469]:
y_test = y[73000:]

In [470]:
scaler = initialize_scaler(X_train=X_train)

In [471]:
X_train_scaled = transform_scaler(scaler, X_train)

In [472]:
X_val_scaled = transform_scaler(scaler, X_val)

In [473]:
X_test_scaled = transform_scaler(scaler, X_test)

In [474]:
X_test_scaled.shape

(4396, 24)

In [475]:
accs = []

In [476]:
accs

[]

In [477]:


scaler = initialize_scaler(X_train=X_train)
X_train_scaled = transform_scaler(scaler, X_train)
X_val_scaled = transform_scaler(scaler, X_val)
X_test_scaled = transform_scaler(scaler, X_test)
model = Sequential()
model.add(layers.Input((24,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
es = EarlyStopping(patience=20, restore_best_weights=True)
model.fit(X_train_scaled, y_train, epochs=200, callbacks=[es], validation_data=[X_val_scaled,y_val])
r = model.evaluate(X_test_scaled[:1000], y_test[:1000])[0]
accs.append(r)
    

Epoch 1/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 1s 504us/step - accuracy: 0.3758 - loss: 1.0960 - val_accuracy: 0.4890 - val_loss: 1.0385
Epoch 2/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step - accuracy: 0.4456 - loss: 1.0565 - val_accuracy: 0.5070 - val_loss: 1.0021
Epoch 3/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 425us/step - accuracy: 0.4848 - loss: 1.0289 - val_accuracy: 0.5230 - val_loss: 0.9843
Epoch 4/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.4913 - loss: 1.0194 - val_accuracy: 0.5550 - val_loss: 0.9633
Epoch 5/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.5087 - loss: 1.0043 - val_accuracy: 0.5570 - val_loss: 0.9581
Epoch 6/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 419us/step - accuracy: 0.5184 - loss: 0.9972 - val_accuracy: 0.5530 - val_loss: 0.9475
Epoch 7/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5141 - loss: 0.9942 - val_accuracy: 0.5650 - val_loss: 0.9535
Epoch 8/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.5157 - loss: 0

688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.5491 - loss: 0.9408 - val_accuracy: 0.5670 - val_loss: 0.9097
Epoch 51/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.5461 - loss: 0.9468 - val_accuracy: 0.5680 - val_loss: 0.9129
Epoch 52/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.5448 - loss: 0.9429 - val_accuracy: 0.5620 - val_loss: 0.9185
Epoch 53/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step - accuracy: 0.5523 - loss: 0.9396 - val_accuracy: 0.5730 - val_loss: 0.8984
Epoch 54/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5451 - loss: 0.9453 - val_accuracy: 0.5590 - val_loss: 0.9125
Epoch 55/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.5507 - loss: 0.9435 - val_accuracy: 0.5610 - val_loss: 0.9083
Epoch 56/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - accuracy: 0.5525 - loss: 0.9385 - val_accuracy: 0.5670 - val_loss: 0.9137
Epoch 57/200
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.5469 - loss: 0.9407

In [478]:
model.evaluate(X_test_scaled, y_val)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 299us/step - accuracy: 0.5639 - loss: 0.8875


[0.89841228723526, 0.5730000138282776]

In [480]:
y.value_counts(normalize=True)

Category_1  Category_0  Category_-1
1           0           0              0.356530
0           0           1              0.341839
            1           0              0.301631
Name: proportion, dtype: float64

In [247]:
model = Sequential()
model.add(layers.Input((24,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))


In [248]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [249]:
es = EarlyStopping(patience=50, restore_best_weights=True)

In [250]:
model.fit(X_train_scaled, y_train, epochs=500, callbacks=[es], validation_data=[X_val_scaled,y_val])

Epoch 1/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 1s 496us/step - accuracy: 0.3734 - loss: 1.0906 - val_accuracy: 0.4840 - val_loss: 1.0209
Epoch 2/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 438us/step - accuracy: 0.4590 - loss: 1.0485 - val_accuracy: 0.5400 - val_loss: 0.9888
Epoch 3/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 472us/step - accuracy: 0.4831 - loss: 1.0250 - val_accuracy: 0.5410 - val_loss: 0.9831
Epoch 4/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - accuracy: 0.4954 - loss: 1.0133 - val_accuracy: 0.5620 - val_loss: 0.9538
Epoch 5/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.5140 - loss: 1.0034 - val_accuracy: 0.5260 - val_loss: 0.9716
Epoch 6/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - accuracy: 0.5129 - loss: 0.9976 - val_accuracy: 0.5540 - val_loss: 0.9583
Epoch 7/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.5247 - loss: 0.9894 - val_accuracy: 0.5580 - val_loss: 0.9578
Epoch 8/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.5196 - loss: 0

688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.5460 - loss: 0.9486 - val_accuracy: 0.5700 - val_loss: 0.9056
Epoch 51/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.5475 - loss: 0.9442 - val_accuracy: 0.5710 - val_loss: 0.9194
Epoch 52/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.5600 - loss: 0.9386 - val_accuracy: 0.5730 - val_loss: 0.9100
Epoch 53/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5467 - loss: 0.9500 - val_accuracy: 0.5780 - val_loss: 0.9078
Epoch 54/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 414us/step - accuracy: 0.5426 - loss: 0.9506 - val_accuracy: 0.5570 - val_loss: 0.9220
Epoch 55/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.5507 - loss: 0.9468 - val_accuracy: 0.5800 - val_loss: 0.9096
Epoch 56/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5424 - loss: 0.9470 - val_accuracy: 0.5770 - val_loss: 0.8960
Epoch 57/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.5521 - loss: 0.9420

688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5560 - loss: 0.9315 - val_accuracy: 0.5980 - val_loss: 0.8969
Epoch 100/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.5589 - loss: 0.9308 - val_accuracy: 0.5760 - val_loss: 0.8948
Epoch 101/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step - accuracy: 0.5573 - loss: 0.9329 - val_accuracy: 0.5860 - val_loss: 0.8962
Epoch 102/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step - accuracy: 0.5566 - loss: 0.9321 - val_accuracy: 0.5570 - val_loss: 0.9253
Epoch 103/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.5562 - loss: 0.9374 - val_accuracy: 0.5810 - val_loss: 0.8903
Epoch 104/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.5571 - loss: 0.9336 - val_accuracy: 0.5990 - val_loss: 0.8896
Epoch 105/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.5649 - loss: 0.9267 - val_accuracy: 0.5910 - val_loss: 0.8912
Epoch 106/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step - accuracy: 0.5570 - loss:

688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.5607 - loss: 0.9239 - val_accuracy: 0.5760 - val_loss: 0.8935
Epoch 149/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step - accuracy: 0.5629 - loss: 0.9216 - val_accuracy: 0.5760 - val_loss: 0.8875
Epoch 150/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step - accuracy: 0.5654 - loss: 0.9249 - val_accuracy: 0.5940 - val_loss: 0.8902
Epoch 151/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5676 - loss: 0.9208 - val_accuracy: 0.5910 - val_loss: 0.8840
Epoch 152/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5619 - loss: 0.9212 - val_accuracy: 0.5730 - val_loss: 0.8983
Epoch 153/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step - accuracy: 0.5683 - loss: 0.9221 - val_accuracy: 0.5950 - val_loss: 0.8835
Epoch 154/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.5654 - loss: 0.9163 - val_accuracy: 0.5920 - val_loss: 0.8779
Epoch 155/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 415us/step - accuracy: 0.5684 - loss:

688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 405us/step - accuracy: 0.5699 - loss: 0.9100 - val_accuracy: 0.5910 - val_loss: 0.8830
Epoch 198/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.5700 - loss: 0.9134 - val_accuracy: 0.5840 - val_loss: 0.8920
Epoch 199/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step - accuracy: 0.5623 - loss: 0.9180 - val_accuracy: 0.5930 - val_loss: 0.8885
Epoch 200/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 400us/step - accuracy: 0.5641 - loss: 0.9152 - val_accuracy: 0.5920 - val_loss: 0.8827
Epoch 201/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.5665 - loss: 0.9149 - val_accuracy: 0.5920 - val_loss: 0.8908
Epoch 202/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 398us/step - accuracy: 0.5644 - loss: 0.9140 - val_accuracy: 0.5910 - val_loss: 0.8861
Epoch 203/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5654 - loss: 0.9172 - val_accuracy: 0.5900 - val_loss: 0.8987
Epoch 204/500
688/688 ━━━━━━━━━━━━━━━━━━━━ 0s 413us/step - accuracy: 0.5693 - loss:

In [251]:
y_train.value_counts(normalize=True)

Category_1  Category_0  Category_-1
1           0           0              0.369227
0           0           1              0.346591
            1           0              0.284182
Name: proportion, dtype: float64

In [262]:
r = model.evaluate(X_val_scaled[:500], y_val[:500])[1]

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - accuracy: 0.6137 - loss: 0.8463


In [239]:
r

0.5839999914169312

In [232]:
model.evaluate(X_val_scaled, y_val)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.5836 - loss: 0.8849


[0.8958861827850342, 0.5839999914169312]

In [155]:
model.save('NEE_model.keras')

In [185]:
model = te.load_weights('NEE_model.keras')

ValueError: A total of 2 objects could not be loaded. Example error message for object <Dense name=dense_28, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(24, 64), Received: value.shape=(27, 64). Target variable: <KerasVariable shape=(24, 64), dtype=float32, path=sequential_7/dense_28/kernel>

List of objects that could not be loaded:
[<Dense name=dense_28, built=True>, <keras.src.optimizers.adam.Adam object at 0x319c5efb0>]

In [265]:
price

,datetime,open,high,low,close,volume
47665,2005-01-03 09:30:00+00:00,5.278,5.304,5.194,5.228,1110640
62557,2005-01-03 10:00:00+00:00,5.229,5.277,5.185,5.244,679280
30227,2005-01-03 10:30:00+00:00,5.245,5.273,5.191,5.223,556400
24130,2005-01-03 11:00:00+00:00,5.223,5.253,5.172,5.219,721040
69934,2005-01-03 11:30:00+00:00,5.220,5.249,5.186,5.216,479680
...,...,...,...,...,...,...
39723,2023-08-31 17:30:00+00:00,65.720,65.723,65.640,65.648,11179
39724,2023-08-31 18:00:00+00:00,65.720,65.723,65.689,65.707,1062
39726,2023-08-31 18:30:00+00:00,65.661,65.723,65.640,65.707,2294171
39727,2023-08-31 19:00:00+00:00,65.661,65.723,65.640,65.707,2293999


In [263]:
key = '0IWDNSUHW5E2KC76'

In [266]:
query = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=NEE&interval=30min&month=2023-09&outputsize=full&apikey={key}'

In [267]:
import requests

In [269]:
results = requests.get(query).json()

In [277]:
price_new = pd.DataFrame(results['Time Series (30min)']).T[::1]

In [279]:
stock = 'NEE'

In [280]:
month = '2023-09'

In [296]:
query = f'https://www.alphavantage.co/query?function=EMA&symbol={stock}&interval=30min&month={month}&time_period=60&series_type=open&apikey={key}'

In [297]:
ema = requests.get(query).json()

In [298]:
r = requests.get(query)
r = r.json()
ema = pd.DataFrame(r['Technical Analysis: EMA']).T

In [299]:
CMO = f'https://www.alphavantage.co/query?function=CMO&symbol={stock}&interval=30min&month={month}&time_period=60&series_type=open&apikey={key}'

In [300]:
query = CMO
r = requests.get(query)
r = r.json()
cmo = pd.DataFrame(r['Technical Analysis: CMO']).T

In [301]:
ATR = f'https://www.alphavantage.co/query?function=ATR&symbol={stock}&interval=30min&month={month}&time_period=60&apikey={key}'

In [302]:
query = ATR
r = requests.get(query)
r = r.json()
atr = pd.DataFrame(r['Technical Analysis: ATR']).T

In [303]:
WILLR = f'https://www.alphavantage.co/query?function=WILLR&symbol={stock}&interval=30min&month={month}&time_period=60&apikey={key}'

query = WILLR
r = requests.get(query)
r = r.json()
willr = pd.DataFrame(r['Technical Analysis: WILLR']).T

In [304]:
query = f'https://www.alphavantage.co/query?function=BBANDS&symbol={stock}&interval=30min&month={month}&time_period=60&series_type=open&apikey={key}'
r = requests.get(query)
r = r.json()
bbands = pd.DataFrame(r['Technical Analysis: BBANDS']).T

In [305]:
query = f"https://www.alphavantage.co/query?function=STOCH&symbol={stock}&interval=30min&month={month}&apikey={key}"
r = requests.get(query)
r = r.json()
stoch = pd.DataFrame(r['Technical Analysis: STOCH']).T

In [306]:
query = f'https://www.alphavantage.co/query?function=OBV&symbol={stock}&interval=30min&month={month}&apikey={key}'
r = requests.get(query)
r = r.json()
obv = pd.DataFrame(r['Technical Analysis: OBV']).T

In [307]:
query = f'https://www.alphavantage.co/query?function=SAR&symbol={stock}&interval=30min&month={month}&apikey={key}'
r = requests.get(query)
r = r.json()
sar = pd.DataFrame(r['Technical Analysis: SAR']).T

In [310]:
query = f'https://www.alphavantage.co/query?function=RSI&symbol={stock}&interval=30min&month={month}&time_period=60&series_type=open&apikey={key}'
r = requests.get(query)
r = r.json()
rsi = pd.DataFrame(r['Technical Analysis: RSI']).T

In [311]:
query = f'https://www.alphavantage.co/query?function=MACD&symbol={stock}&interval=30min&month={month}&time_period=60&series_type=open&apikey={key}'
r = requests.get(query)
r = r.json()
macd = pd.DataFrame(r['Technical Analysis: MACD']).T

In [381]:
df = ema.merge(stoch, left_index=True, right_index=True, how = 'outer')

In [382]:

df = df.merge(bbands, left_index=True, right_index=True, how = 'outer')
df = df.merge(willr, left_index=True, right_index=True, how = 'outer')
df = df.merge(atr, left_index=True, right_index=True, how = 'outer')
df = df.merge(cmo, left_index=True, right_index=True, how = 'outer')
df = df.merge(macd, left_index=True, right_index=True, how = 'outer')
df = df.merge(rsi, left_index=True, right_index=True, how = 'outer')
df = df.merge(obv, left_index=True, right_index=True, how = 'outer')
df = df.merge(sar, left_index=True, right_index=True, how = 'outer')

In [383]:
df

,EMA,SlowK,SlowD,Real Upper Band,Real Middle Band,Real Lower Band,WILLR,ATR,CMO,MACD,MACD_Signal,MACD_Hist,RSI,OBV,SAR
2023-09-01 04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0000,NaN
2023-09-01 04:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.0000,65.6400
2023-09-01 05:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.0000,65.6430
2023-09-01 06:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.0000,65.6460
2023-09-01 07:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,694.0000,65.6489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-29 17:30,57.5599,43.5078,47.8598,59.8098,57.2332,54.6567,-89.0488,0.4462,-21.8770,-0.1713,-0.1789,0.0075,39.0615,10364303.0000,57.5711
2023-09-29 18:00,57.5207,35.5731,41.8192,59.7116,57.1839,54.6563,-89.0488,0.4406,-21.8601,-0.1630,-0.1757,0.0127,39.0700,10364303.0000,57.5237
2023-09-29 18:30,57.4811,52.3837,43.8215,59.5818,57.1282,54.6746,-89.0488,0.4346,-22.1294,-0.1585,-0.1723,0.0137,38.9353,10364303.0000,57.4777
2023-09-29 19:00,57.4428,52.9225,46.9598,59.4401,57.0717,54.7033,-90.1220,0.4287,-22.1294,-0.1532,-0.1685,0.0152,38.9353,8204866.0000,57.4331


In [315]:
macro

,int64_field_0,timestamp,interest_rate,gdp,inflation
322896,327215,2005-01-01 00:00:00+00:00,1.97,3867.076,3.392747
322897,327214,2005-01-01 00:30:00+00:00,1.97,3867.076,3.392747
322898,327213,2005-01-01 01:00:00+00:00,1.97,3867.076,3.392747
322899,327212,2005-01-01 01:30:00+00:00,1.97,3867.076,3.392747
322900,327211,2005-01-01 02:00:00+00:00,1.97,3867.076,3.392747
...,...,...,...,...,...
77419,4,2023-08-31 21:30:00+00:00,5.33,5642.697,4.116338
77420,3,2023-08-31 22:00:00+00:00,5.33,5642.697,4.116338
77421,2,2023-08-31 22:30:00+00:00,5.33,5642.697,4.116338
77422,1,2023-08-31 23:00:00+00:00,5.33,5642.697,4.116338


In [316]:
url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval=daily&apikey={key}'
r = requests.get(url)
data = r.json()

In [321]:
df_interest_rate = pd.DataFrame(data['data'])

In [348]:
df_interest_rate = df_interest_rate[334:364]

In [361]:
url = f'https://www.alphavantage.co/query?function=REAL_GDP&interval=quarterly&apikey={key}'
r = requests.get(url)
data = r.json()

In [362]:
df_gdp = pd.DataFrame(data['data'])

In [366]:
df_interest_rate['gdp'] = 5642.697

/var/folders/7w/19z1p17n6vb_v1k8gw86dgm40000gn/T/ipykernel_15439/966274146.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest_rate['gdp'] = 5642.697


In [368]:
url = 'https://www.alphavantage.co/query?function=INFLATION&apikey=demo'
r = requests.get(url)
data = r.json()

In [372]:
pd.DataFrame(data['data'])

,date,value
0,2023-01-01,4.11633838374488
1,2022-01-01,8.00279982052121
2,2021-01-01,4.69785886363742
3,2020-01-01,1.23358439630638
4,2019-01-01,1.81221007526012
...,...,...
59,1964-01-01,1.27891156462591
60,1963-01-01,1.23966942148761
61,1962-01-01,1.19877334820149
62,1961-01-01,1.07072414764754


In [373]:
df_interest_rate['inflation'] = 4.11633838374488

/var/folders/7w/19z1p17n6vb_v1k8gw86dgm40000gn/T/ipykernel_15439/4040150394.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest_rate['inflation'] = 4.11633838374488


In [375]:
df_interest_rate['int64_field_0'] = 0

/var/folders/7w/19z1p17n6vb_v1k8gw86dgm40000gn/T/ipykernel_15439/2943308875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest_rate['int64_field_0'] = 0


In [377]:
df_interest_rate['date'] = pd.to_datetime(df_interest_rate['date'])

/var/folders/7w/19z1p17n6vb_v1k8gw86dgm40000gn/T/ipykernel_15439/1546073327.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_interest_rate['date'] = pd.to_datetime(df_interest_rate['date'])


In [379]:
df_interest_rate = df_interest_rate[['date','value', 'gdp', 'inflation','int64_field_0']]

In [380]:
df_interest_rate

,date,value,gdp,inflation,int64_field_0
334,2023-09-30,5.33,5642.697,4.116338,0
335,2023-09-29,5.33,5642.697,4.116338,0
336,2023-09-28,5.33,5642.697,4.116338,0
337,2023-09-27,5.33,5642.697,4.116338,0
338,2023-09-26,5.33,5642.697,4.116338,0
339,2023-09-25,5.33,5642.697,4.116338,0
340,2023-09-24,5.33,5642.697,4.116338,0
341,2023-09-23,5.33,5642.697,4.116338,0
342,2023-09-22,5.33,5642.697,4.116338,0
343,2023-09-21,5.33,5642.697,4.116338,0


In [ ]:
["datetime", "interest_rate", "GDP", "inflation"]

In [385]:
price_new

,1. open,2. high,3. low,4. close,5. volume
2023-09-29 19:30:00,56.3620,56.3750,56.1090,56.3610,477
2023-09-29 19:00:00,56.3130,56.3750,56.2950,56.3170,2159437
2023-09-29 18:30:00,56.3130,56.3750,56.2950,56.3610,2159326
2023-09-29 18:00:00,56.3620,56.3750,56.2660,56.3610,1438
2023-09-29 17:30:00,56.3600,56.3750,56.3050,56.3610,226
...,...,...,...,...,...
2023-09-01 07:00:00,65.8670,66.0080,65.8370,65.8740,563
2023-09-01 06:30:00,65.8570,65.8610,65.6600,65.6680,3
2023-09-01 05:30:00,65.9360,65.9390,65.9150,65.9230,2
2023-09-01 04:30:00,65.9360,65.9390,65.9150,65.9230,59


In [390]:
price_new

,1. open,2. high,3. low,4. close,5. volume
2023-09-29 19:30:00,56.3620,56.3750,56.1090,56.3610,477
2023-09-29 19:00:00,56.3130,56.3750,56.2950,56.3170,2159437
2023-09-29 18:30:00,56.3130,56.3750,56.2950,56.3610,2159326
2023-09-29 18:00:00,56.3620,56.3750,56.2660,56.3610,1438
2023-09-29 17:30:00,56.3600,56.3750,56.3050,56.3610,226
...,...,...,...,...,...
2023-09-01 07:00:00,65.8670,66.0080,65.8370,65.8740,563
2023-09-01 06:30:00,65.8570,65.8610,65.6600,65.6680,3
2023-09-01 05:30:00,65.9360,65.9390,65.9150,65.9230,2
2023-09-01 04:30:00,65.9360,65.9390,65.9150,65.9230,59


In [388]:
df_interest_rate

,date,value,gdp,inflation,int64_field_0
334,2023-09-30,5.33,5642.697,4.116338,0
335,2023-09-29,5.33,5642.697,4.116338,0
336,2023-09-28,5.33,5642.697,4.116338,0
337,2023-09-27,5.33,5642.697,4.116338,0
338,2023-09-26,5.33,5642.697,4.116338,0
339,2023-09-25,5.33,5642.697,4.116338,0
340,2023-09-24,5.33,5642.697,4.116338,0
341,2023-09-23,5.33,5642.697,4.116338,0
342,2023-09-22,5.33,5642.697,4.116338,0
343,2023-09-21,5.33,5642.697,4.116338,0


In [425]:
import tensorflow as tf

In [426]:
model = tf.keras.models.load_model('NEE_model.keras')

In [428]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 64)             │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 10)             │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 3)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,707 (49.64 KB)

 Trainable params: 4,235 (16.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 8,472 (33.10 KB)

In [405]:
df.reset_index(inplace=True)

In [408]:
df.columns = ['datetime', 'EMA', 'SlowK', 'SlowD', 'Real Upper Band', 'Real Middle Band',
       'Real Lower Band', 'WILLR', 'ATR', 'CMO', 'MACD', 'MACD_Signal',
       'MACD_Hist', 'RSI', 'OBV', 'SAR']

In [415]:
price_new = price_new.drop(columns='index')

In [ ]:
model.evaluate()

In [431]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [433]:
price_new['datetime'] = pd.to_datetime(price_new['datetime'])

In [434]:
price_new

,datetime,open,high,low,close,volume
0,2023-09-29 19:30:00,56.3620,56.3750,56.1090,56.3610,477
1,2023-09-29 19:00:00,56.3130,56.3750,56.2950,56.3170,2159437
2,2023-09-29 18:30:00,56.3130,56.3750,56.2950,56.3610,2159326
3,2023-09-29 18:00:00,56.3620,56.3750,56.2660,56.3610,1438
4,2023-09-29 17:30:00,56.3600,56.3750,56.3050,56.3610,226
...,...,...,...,...,...,...
614,2023-09-01 07:00:00,65.8670,66.0080,65.8370,65.8740,563
615,2023-09-01 06:30:00,65.8570,65.8610,65.6600,65.6680,3
616,2023-09-01 05:30:00,65.9360,65.9390,65.9150,65.9230,2
617,2023-09-01 04:30:00,65.9360,65.9390,65.9150,65.9230,59


In [443]:
price_new.merge(df, right_on = 'datetime', left_on = 'datetime')

,datetime,open,high,low,close,volume,EMA,SlowK,SlowD,Real Upper Band,...,Real Lower Band,WILLR,ATR,CMO,MACD,MACD_Signal,MACD_Hist,RSI,OBV,SAR
0,2023-09-29 19:30:00,56.3620,56.3750,56.1090,56.3610,477,57.4073,76.2273,60.5112,59.3022,...,54.7344,-89.0488,0.4260,-21.6957,-0.1434,-0.1635,0.0200,39.1522,8205343.0000,57.3898
1,2023-09-29 19:00:00,56.3130,56.3750,56.2950,56.3170,2159437,57.4428,52.9225,46.9598,59.4401,...,54.7033,-90.1220,0.4287,-22.1294,-0.1532,-0.1685,0.0152,38.9353,8204866.0000,57.4331
2,2023-09-29 18:30:00,56.3130,56.3750,56.2950,56.3610,2159326,57.4811,52.3837,43.8215,59.5818,...,54.6746,-89.0488,0.4346,-22.1294,-0.1585,-0.1723,0.0137,38.9353,10364303.0000,57.4777
3,2023-09-29 18:00:00,56.3620,56.3750,56.2660,56.3610,1438,57.5207,35.5731,41.8192,59.7116,...,54.6563,-89.0488,0.4406,-21.8601,-0.1630,-0.1757,0.0127,39.0700,10364303.0000,57.5237
4,2023-09-29 17:30:00,56.3600,56.3750,56.3050,56.3610,226,57.5599,43.5078,47.8598,59.8098,...,54.6567,-89.0488,0.4462,-21.8770,-0.1713,-0.1789,0.0075,39.0615,10364303.0000,57.5711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,2023-09-01 07:00:00,65.8670,66.0080,65.8370,65.8740,563,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,694.0000,65.6489
615,2023-09-01 06:30:00,65.8570,65.8610,65.6600,65.6680,3,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,131.0000,65.6460
616,2023-09-01 05:30:00,65.9360,65.9390,65.9150,65.9230,2,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.0000,65.6430
617,2023-09-01 04:30:00,65.9360,65.9390,65.9150,65.9230,59,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.0000,65.6400


In [437]:
df_interest_rate

,date,value,gdp,inflation,int64_field_0
334,2023-09-30,5.33,5642.697,4.116338,0
335,2023-09-29,5.33,5642.697,4.116338,0
336,2023-09-28,5.33,5642.697,4.116338,0
337,2023-09-27,5.33,5642.697,4.116338,0
338,2023-09-26,5.33,5642.697,4.116338,0
339,2023-09-25,5.33,5642.697,4.116338,0
340,2023-09-24,5.33,5642.697,4.116338,0
341,2023-09-23,5.33,5642.697,4.116338,0
342,2023-09-22,5.33,5642.697,4.116338,0
343,2023-09-21,5.33,5642.697,4.116338,0
